# Население беларуси

In [1]:
#r "nuget:XPlot.Plotly, 3.0.1"
#r "nuget:HtmlAgilityPack, 1.11.24.0"
    
using System.Net;
using System.Text;
using System.Globalization;
using XPlot.Plotly;
using HtmlAgilityPack;
using System.Data;

Installing package XPlot.Plotly, version 3.0.1.........................done!

Successfully added reference to package XPlot.Plotly, version

Installing package HtmlAgilityPack, version 1.11.24.0...done!

Successfully added reference to package HtmlAgilityPack, version

In [2]:
// сглаживающая функция
Func<IEnumerable<double>, int, IEnumerable<double>> smoothingFunc = (input, times/*гладкость*/) =>
{
    var result = input.ToList();//clone
    for (int j = 0; j < times; j++) 
    {
        result = result.Select((x, i) => result.Skip(i-1).Take(3).Average()).ToList();
    }
    
    return result;
};

In [3]:
// наша модель данных
public class BelarusPipleStat
{
    public BelarusPipleStat(string[] prms)
    {
        Year = Int32.Parse(prms[0]);
        Total = Int32.Parse(prms[1]);
        New = Double.Parse(prms[3]);
        
        var deths = 0D;
        if(Double.TryParse(prms[5], out deths))
            Deths = deths;
    }    
    
    public int Year {get;set;}        
    public int Total {get;set;}    
    public double New {get;set;}    
    public double? Deths {get;set;}   
}

In [4]:
// стянем данные из википедии
var web = new HtmlWeb();
var doc = web.Load("https://ru.wikipedia.org/wiki/Население_Белоруссии");
var tableNode2 = doc.DocumentNode.SelectSingleNode("/html/body/div[3]/div[3]/div[4]/div/table[3]");

var headers = tableNode2.SelectNodes(".//tr/th");
var dataList = new List<BelarusPipleStat>();

foreach (var row in tableNode2.SelectNodes(".//tr[td]"))
{
    dataList.Add(new BelarusPipleStat(row.SelectNodes(".//td//text()").Select(td =>td.InnerText.Replace("&#160;", "").Replace("&#nbsp;", "").Replace(" ", "")).ToArray()));
}    

In [5]:
display(dataList);

index,Year,Total,New,Deths
0,1950,7745000,197164,61944
1,1951,7765000,198200,61200
2,1952,7721000,191100,64200
3,1953,7690000,176200,61900
4,1954,7722000,193100,61600
5,1955,7804000,194345,57966
6,1956,7880000,199500,56000
7,1957,7936000,200800,58000
8,1958,8009000,207700,53700
9,1959,8055714,204600,62600


In [6]:
// отобразим численность населения с 93го года
var belarusPopulation = dataList.Where(x=>x.Year>=1993);
var belarusPipleTotalPlot = new Graph.Scatter(){x=belarusPopulation.Select(x=>x.Year), y=belarusPopulation.Select(x=>x.Total), name="Численость"};
display(Chart.Plot(new [] {belarusPipleTotalPlot}));

In [7]:
// отобразим прирост/убывание немного сгладив
var belarusPopulationNewPlot = new Graph.Scatter(){x=belarusPopulation.Select(x=>x.Year), y=smoothingFunc(belarusPopulation.Select(x=>x.New), 5), name="Родилось"};
var belarusPopulationDeathPlot = new Graph.Scatter(){x=belarusPopulation.Select(x=>x.Year), y=smoothingFunc(belarusPopulation.Where(x=>x.Deths.HasValue).Select(x=>x.Deths.Value), 5), name="Умерло"};
display(Chart.Plot(new [] { belarusPopulationNewPlot, belarusPopulationDeathPlot }));